# Game Tournament
## Description
### Inspiration / Source / Backstory
A friend of mine likes the videogame Hearthstone, and he had a question about the odds of winning a prize in an event they're running called "Brawliseum".
Here's a link to the description on the fan wiki page:
https://hearthstone.fandom.com/wiki/Heroic_Brawliseum

I'm going to structure the problem in a generic way and look at how to solve it.

### Problem Statement
There is a game tournament in which each *admission* allows a player to play *games* until they either achieve $n$ wins, or $k$ losses, whichever comes first.  
Players may re-enter, but each admission has a cost $c$.  
Furthermore you may assume each game takes an average time $t$ to play.  
Assume an average probability of winning a single game $p$.

### Values for the Backstory
- $p = 50\%$ (since for each winner, there is also a loser)
- $n = 12$
- $k = 3$
- $c = \$ 10$
- $t = 10 \mathrm{mins}$

### Questions
- What is the probability, $\mathrm{P}$, of winning the prize in a single admission?
- What is the expected number of admissions, $\mathrm{N}$, to win the prize?
- What is the expected total cost, $\mathrm{C}$, to win the prize?
- What is the expected number of games, $\mathrm{G}_{1}$, played in a single admission?
- What is the expected number of games, $\mathrm{G}$, played to win the prize?
- What is the expected total time, $\mathrm{T}$, it will take to win the prize?

## Numerical Solution: Monte Carlo Simulation
Probabily the easiest way to solve this problem with coding, as well as a good way to check our exact solutions, is to simulate it directly and track the stats on the results.

#### Simulation Code

In [1]:
import random

Using the values for the backstory 

In [2]:
n = 12
k = 3
p = 0.5
t = 10
c = 10

Obviously the more samples we collect the lower our error will be. I won't do any analysis on expected error or variance here.

In [3]:
samples = 100*1000

In [4]:
total_admissions = 0
total_cost = 0
total_games = 0
total_time = 0

total_prizes = 0
for i in range(samples):
    while True:
        total_admissions += 1
        total_cost += c
        wins, losses = 0,0
        while wins < n and losses < k:
            if random.random() > p:
                wins += 1
            else:
                losses += 1
            total_games += 1
            total_time += t
        if wins == n:
            total_prizes += 1
            break

#### Computing P from Simulation Stats
The probability of winning the prize in a single admissions would be ratio of the total number of prizes won in our simulation divided by the total number of admissions: i.e. normalizing the prizes by admissions.  
This gives a result of $\mathrm{P} \approx 0.64 \%$

In [5]:
P = total_prizes / total_admissions
P

0.006475016117933872

#### Computing N from Simulation Stats
The average number of admissions per win of the prize is simply the total number of admissions divided by the total number of prizes won: i.e. th reciprocal of $\mathrm{P}$.  
This gives a result of $\mathrm{N} \approx 156 $

In [6]:
N = total_admissions / total_prizes
N

154.43977

#### Computing C from Simulation Stats
The average cost to win of the prize is similar to the admissions.  
This gives a result of $\mathrm{C} \approx \$ 1558 $

In [7]:
C = total_cost / total_prizes
C

1544.3977

#### Computing G1 from Simulation Stats
The average games played in a single admission can be computed by dividing the total number of games by the total number of admissions.  
This gives a result of $\mathrm{G}_{1} \approx 6 $

In [8]:
G1 = total_games / total_admissions
G1

5.983737932269648

#### Computing G from Simulation Stats
The average games played to win the prize can be computed by dividing the total number of games by the total number of prizes won.  
This gives a result of $\mathrm{G} \approx 932 $

In [9]:
G = total_games / total_prizes
G

924.12711

#### Computing T from Simulation Stats
The average time to win the prize can be computed by dividing the total time by the total number of prizes won.  
This gives a result of $
\mathrm{T} 
\approx 9325 \, \mathrm{mins} 
\approx 155 \, \mathrm{hours} 
\approx 6 \,\mathrm{days} $

In [10]:
T = total_time / total_prizes
T

9241.2711

In [11]:
T / 60

154.021185

In [12]:
T / 60 / 24

6.417549375

## Recursive Solution: Dynamic Programming
https://en.wikipedia.org/wiki/Dynamic_programming  
https://en.wikipedia.org/wiki/Memoization  

For an exact solution we need to study the problem more analytically.  
We can define the probability of winning the prize on a single tournament entry recursively as follows:

$$ 
\mathrm{P}(n,k) = 
\begin{cases} 
    1 & n = 0 \\
    0 & k = 0 \\
    p \cdot \mathrm{P}(n-1,k) + (1-p) \cdot \mathrm{P}(n,k-1) & n,k \gt 0 \\
\end{cases}
$$

Where
- $ \mathrm{P}(n,k) $ is the probability of getting $n$ successes before getting $k$ failures.
- $ p $ is the probability of success on a single game.

We can therefore solve this problem using dynamic programming, specifically memoization or cacheing of sub-solutions so as to not recompute them.  
This gives an asymptotic complexity of $O(nk)$ in time and memory.

#### P with Dynamic Programming Code
With the numbers from our backstory we get a result of $\mathrm{P}(12,3) = 0.647\%$, which is inline with our numerical approximation.

In [13]:
from functools import cache

In [14]:
p = 0.5
@cache
def P(n,k):
    if n == 0:
        return 1.
    if k == 0:
        return 0.
    return p * P(n-1,k) + (1-p) * P(n,k-1)
P(12,3)

0.0064697265625

#### Solving for N from P
Now for the number of admissions required, we can use the classic fact from probablility that the expected number of attempts is the reciprocal of the probability.
$$ \mathrm{N}(n,k) = \frac{1}{\mathrm{P}(n,k)} $$

Where
- $ \mathrm{N}(n,k) $ is the expected number of admissions to get $n$ successes before getting $k$ failures.
- $ \mathrm{P}(n,k) $ is the probability of getting $n$ successes before getting $k$ failures.

With the numbers from our backstory we get a result of $\mathrm{N}(12,3) = 154.566$, which is inline with our numerical approximation.

In [15]:
N = 1/P(12,3)
N

154.56603773584905

#### Solving for C from N
Our equation for cost comes from a straight-forward proportionality
$$ \mathrm{C} = c \mathrm{N} = \frac{c}{\mathrm{P}} $$

Where
- $ \mathrm{C} $ is the expected total cost to win the prize.
- $ c $ is the cost of a single admission.
- $ \mathrm{N} $ is the expected number of admissions to win the prize.
- $ \mathrm{P} $ is the probability of winning on a single admission.

With the numbers from our backstory we get a result of $\mathrm{C} = \$ 1545.66$, which is inline with our numerical approximation.

In [16]:
c = 10
C = c * N
C

1545.6603773584905

#### Dynamic Programming for G1, G and T
We can solve for $G_{1}$, $G$ and $T$, as well with a recursive definition taking into account the number of games played:

$$ 
\mathrm{G}_{1}(n,k) = 
\begin{cases} 
    n_{0} + k_{0} - k & n = 0 \\
    n_{0} + k_{0} - n & k = 0 \\
    p \cdot \mathrm{G}_{1}(n-1,k) + (1-p) \cdot \mathrm{G}_{1}(n,k-1) & n,k \gt 0 \\
\end{cases}
$$

Where
- $ \mathrm{G}_{1}(n,k) $ is the expected number of games played in an admission, given there are $n$ successes and $k$ failures remaining in the current admission.
- $n_{0}$ is the number of successes in an admission required to win the prize.
- $k_{0}$ is the number of failures in an admission resulting in requiring a re-try.
- $p$ is the probability of success on a single game.

We can therefore solve this problem using dynamic programming with an asymptotic complexity of $O(nk)$ in time and memory.

#### G1 with Dynamic Programming Code
With the numbers from our backstory we get a result of $\mathrm{G}_1(12,3) = 5.983$, which is inline with our numerical approximation.

In [17]:
n0 = 12
k0 = 3
p = 0.5
@cache
def G1(n,k):
    if n == 0 or k == 0:
        return n0 + k0 - n - k
    return p * G1(n-1,k) + (1-p) * G1(n,k-1)
G1(n0,k0)

5.983154296875

#### Solving for G from G1 and N
If we know the expected number of admissions to win the prize, $\mathrm{N}$, and the expected number of games played per admission, $\mathrm{G}_{1}$, then logically we should be able to compute the expected number of games played to win the prize, $\mathrm{G}$, with the following equation:

$$ \mathrm{G} = \mathrm{G}_{1} \mathrm{N} = \frac{\mathrm{G}_{1}}{P} $$

Where
- $ \mathrm{G} $ is the expected total number of games played.
- $ \mathrm{G}_{1} $ is the expected number of games played in a single admission.
- $ \mathrm{N} $ is the expected number of admissions to win the prize.
- $ \mathrm{P} $ is the probability of winning on a single admission.

Is this legitimate though? The answer is yes, because each admission is independent from eachother: the expected number of games for each admission does not change no matter how many admissions we enter.

With the numbers from our backstory we get a result of $\mathrm{G} = 924.792$, which is inline with our numerical approximation.

In [18]:
G = G1(n0,k0) * N
G

924.7924528301886

#### Solving for T from G
Our equation for time comes from a straight-forward proportionality
$$ \mathrm{T} = t \mathrm{G} = t \mathrm{G}_{1} \mathrm{N} = \frac{t \mathrm{G}_{1}}{P}$$

Where
- $ \mathrm{T} $ is the expected total time to win the prize.
- $ t $ is the time to play a single game.
- $ \mathrm{G} $ is the expected total number of games played.
- $ \mathrm{G}_{1} $ is the expected number of games played in a single admission.
- $ \mathrm{N} $ is the expected number of admissions to win the prize.
- $ \mathrm{P} $ is the probability of winning on a single admission.

With the numbers from our backstory we get a result of $\mathrm{T} = 9196.736 \mathrm{mins} 
= 6 \mathrm{days} \, 9 \mathrm{hours} \, 16 \mathrm{mins}$, which is inline with our numerical approximation.

In [19]:
t = 10
T = t * G
T

9247.924528301886

In [20]:
minutes = T % 60
hours = int(T-minutes) // 60 % 24
days = int(T-minutes-60*hours) // 60 // 24
print(f"{days} days {hours} hours and {minutes} minutes")

6 days 10 hours and 7.924528301886312 minutes


## Recursive Solution 2: Linear Systems
In the same way as we defined the the probability of winning in a single entry recursively, we can define the expected number of admissions to win the prize recursively as follows:

$$ 
\mathrm{N}(n,k) = 
\begin{cases} 
    1 & n = 0 \\
    1 + \mathrm{N}(n_{0},k_{0}) & k = 0 \\
    p \cdot \mathrm{N}(n-1,k) + (1-p) \cdot \mathrm{N}(n,k-1) & n,k \gt 0 \\
\end{cases}
$$


Where
- $\mathrm{N}(n,k)$ is the expected number of admissions required to win the prize, given there are $n$ successes and $k$ failures remaining in the current admission.
- $n_{0}$ is the number of successes in an admission required to win the prize.
- $k_{0}$ is the number of failures in an admission resulting in requiring a re-try.
- $p$ is the probability of success on a single game.

Now this problem has a cycle in its recursive definition, so we can't use dynamic programming, however we fortunately do have a linear system, and for our relatively low number of states should be able to solve this without too much trouble!

We need values and have equations for $\mathrm{N}(n,k)$ for $n,k \in [0,n_{0}] \times [0,k_{0}]$.  
Which means we have $(n_{0}+1)(k_{0}+1)$ equations and variables, and therefore a $(n_{0}+1)(k_{0}+1) \times (n_{0}+1)(k_{0}+1)$ square matrix.

However it should be noted that this approach takes $O(n^{2}k^{2})$ space complexity, and $O(n^{3}k^{3})$ time complexity with general linear system algorithms (like Gauss-Jordan), however since we have a sparse, and almost banded matrix, we should be able to solve it in $O(nk)$ time and space. Furthermore with the matrix division it should also be noted that the linear stability likely isn't as good, and same with the expected error.

#### N with Linear Systems Code
With the numbers from our backstory we get a result of $\mathrm{N} = 154.566$, which matches our previous calculations.

In [21]:
import numpy as np

In [22]:
n0 = 12
k0 = 3
p = 0.5

In [23]:
size = (n0+1) * (k0+1)
N_mat = np.eye(size)
N_vec = np.zeros(size)
for n in range(n0+1):
    for k in range(k0+1):
        if n == 0:
            N_vec[n*(k0+1) + k] = 1
        elif k == 0:
            N_vec[n*(k0+1) + k] = 1
            N_mat[n*(k0+1) + k][n0*(k0+1) + k0] = -1
        else:
            N_mat[n*(k0+1) + k][(n-1)*(k0+1) + k] = -p
            N_mat[n*(k0+1) + k][n*(k0+1) + k-1] = -(1-p)

In [24]:
N = np.linalg.solve(N_mat, N_vec)[-1]
N

154.56603773584905

#### Solving for G with linear systems
We can define the time taken to win the prize on a single tournament entry recursively as follows:

$$ 
\mathrm{G}(n,k) = 
\begin{cases} 
    0 & n = 0 \\
    \mathrm{G}(n_{0},k_{0}) & k = 0 \\
    1 + p \cdot \mathrm{G}(n-1,k) + (1-p) \cdot \mathrm{G}(n,k-1) & n,k \gt 0 \\
\end{cases}
$$


Where
- $\mathrm{G}(n,k)$ is the expected number of games played to win the prize, given there are $n$ successes and $k$ failures remaining in the current admission.
- $n_{0}$ is the number of successes in an admission required to win the prize.
- $k_{0}$ is the number of failures in an admission resulting in requiring a re-try.
- $p$ is the probability of success on a single game.

#### G with Linear Systems Code
With the numbers from our backstory we get a result of $\mathrm{G} = 924.792$, which matches our previous calculations.

In [25]:
n_0 = 12
k_0 = 3
p = 0.5

In [26]:
size = (n_0+1) * (k_0+1)
G_mat = np.eye(size)
G_vec = np.zeros(size)
for n in range(n_0+1):
    for k in range(k_0+1):
        if n == 0:
            pass
        elif k == 0:
            G_mat[n*(k_0+1) + k][n_0*(k_0+1) + k_0] = -1
        else:
            G_mat[n*(k_0+1) + k][(n-1)*(k_0+1) + k] = -p
            G_mat[n*(k_0+1) + k][n*(k_0+1) + k-1] = -(1-p)
            G_vec[n*(k_0+1) + k] = 1

In [27]:
G = np.linalg.solve(G_mat, G_vec)[-1]
G

924.7924528301887

## Analytic Solution: Stars and Bars
https://en.wikipedia.org/wiki/Stars_and_bars_(combinatorics)

Now we've solved this problem with a numerical approximation, as well as computed exact solutions recursively, is there a way we could have solved this problem in an analytical close-form formula?

Turns out the answer is yes!

The way so solve this problem analytically is to sum the probabilities of getting $n$ wins and less than $k$ losses in an admission.

The probability of getting a particular ordered string of $n$ wins and $k$ losses in an admission is $p^{n}(1-p)^{k}$. So we needed to multiply this probability by the number of ways to get $n$ wins and $k$ losses to get the unordered probability.

The "stars-and-bars trick" solves combinatorial problems of the form "How many ways are there to arrange $n$ stars and $k$ bars", or in our case "How many ways are there to get $n$ wins and $k$ losses in an admission".

The number of arrangements is given in terms of the binomial coefficient, $ \binom{n+k}{k} $, which makes sense: if we have $n+k$ slots for stars or bars, and we need to choose $k$ of them to be bars.

The last thing to remember is that we must end of a win, therefore in our star-and-bars we will fix one win at the end, and so only consider $n-1$ in counting our arrangements.

Putting this all together in a sum yields:

$$ \mathrm{P}(n,k) 
= \sum_{j=0}^{k-1} p^{n} (1-p)^{j} \binom{n-1+j}{j} 
= \frac{p^{n}}{(n-1)!} \sum_{j=0}^{k-1} (1-p)^{j} \frac{(n-1+j)!}{j!}
$$

The time complexity is technically not improved because of the factorials in the loops, $O(k^{2})$, but the space complexity has now been eliminated, $O(1)$. Also I would expect the numerical stability of this formula to be much lower than the dynamic programming approach.

#### Computing P Analytically
With the numbers from our backstory we get a result of $\mathrm{P} = 0.647 \%$, which matches our previous calculations.

In [28]:
import math

In [29]:
n = 12
k = 3
p = 0.5

In [30]:
P = sum(p**n * (1-p)**j * math.comb(n-1+j, j) for j in range(k))
P

0.0064697265625

#### Solving for N from P
We can use the same equation for $\mathrm{N}$ as before:
$$ \mathrm{N} = \frac{1}{\mathrm{P}} $$

With the numbers from our backstory we get a result of $\mathrm{N} = 154.566$, which matches our previous calculations.

In [31]:
N = 1/P
N

154.56603773584905

#### Solving for C from N
Again, same as before:
$$ \mathrm{C} = c \mathrm{N} = \frac{c}{\mathrm{P}} $$

With the numbers from our backstory we get a result of $\mathrm{N} = 1545.66$, which matches our previous calculations.

In [32]:
c = 10
C = c*N
C

1545.6603773584905

#### Computing G1 Analytically
We can use a similar approach to compute $\mathrm{G}_{1}$, since we now know how to compute the probability of getting a given number of successes and failure in an entry, we can weight the cases of different numbers of games being played. Note we have to include both the cases where the prize is won and when it isn't.

$$ \mathrm{G}_{1}(n,k) 
= \sum_{j=0}^{k-1} (n+j) p^{n} (1-p)^{j} \binom{n-1+j}{j} 
+ \sum_{j=0}^{n-1} (j+k) p^{j} (1-p)^{k} \binom{j+k-1}{k-1}
$$

Factoring out constant terms from the summations:

$$ \mathrm{G}_{1}(n,k) 
= \frac{p^{n}}{(n-1)!} \sum_{j=0}^{k-1} (n+j) (1-p)^{j} \frac{(n-1+j)!}{j!} 
+ \frac{(1-p)^{k}}{(k-1)!} \sum_{j=0}^{n-1} (j+k) p^{j} \frac{(j+k-1)!}{j!}
$$

Again, the time complexity is technically not improved because of the factorials in the loops, $O(nk)$, but the space complexity has now been eliminated, $O(1)$. Also I would expect the numerical stability of this formula to be much lower than the dynamic programming approach.

With the numbers from our backstory we get a result of $\mathrm{G}_{1} = 5.983$, which matches our previous calculations.

In [33]:
G1 = sum( (n+j) * p**n * (1-p)**j * math.comb(n-1 + j, j) for j in range(k)) \
    + sum( (j+k) * p**j * (1-p)**k * math.comb(j + k-1, k-1) for j in range(n))
G1

5.983154296875

#### Solving for G from G1 and N
We can use the same equation for $\mathrm{G}$ as before:
$$ \mathrm{G} = \mathrm{G}_{1} \mathrm{N} = \frac{\mathrm{G}_{1}}{P} $$

With the numbers from our backstory we get a result of $\mathrm{G} = 924.792$, which matches our previous calculations.

In [34]:
G = G1 * N
G

924.7924528301886

#### Solving for T from G
We can use the same equation for $\mathrm{T}$ as before:
$$ \mathrm{T} = t \mathrm{G} = t \mathrm{G}_{1} \mathrm{N} = \frac{t \mathrm{G}_{1}}{P}$$


With the numbers from our backstory we get a result of $\mathrm{T} = 9247.925 \mathrm{mins} 
= 6 \mathrm{days} \, 10 \mathrm{hours} \, 8 \mathrm{mins}$, which matches our previous calculations.

In [35]:
t = 10
T = t * G
T

9247.924528301886

In [36]:
minutes = T % 60
hours = int(T-minutes) // 60 % 24
days = int(T-minutes-60*hours) // 60 // 24
print(f"{days} days {hours} hours and {minutes} minutes")

6 days 10 hours and 7.924528301886312 minutes


## Result Table
A nice and useful way to present the results is in a table mapping the main dependent variable, the probability of success on a single game $p$, to the answers.  
We'll keep $n=12$ and $k=3$ fixed, along with $t=\$10$ and $c=10 \, \mathrm{mins} $

In [37]:
import numpy as np
import pandas as pd
import math

In [38]:
dp = 0.05

n = 12
k = 3
t = 10
c = 10

In [39]:
p = np.arange(1+dp, step=dp)

P = sum(p**n * (1-p)**j * math.comb(n-1+j, j) for j in range(k))
N = 1/P
C = c*N
G1 = sum( (n+j) * p**n * (1-p)**j * math.comb(n-1 + j, j) for j in range(k)) \
    + sum( (j+k) * p**j * (1-p)**k * math.comb(j + k-1, k-1) for j in range(n))
G = G1 * N
T = t * G

/var/folders/7g/891d6t0d7hb5d120t4hn68mr0000gn/T/ipykernel_38541/2966935887.py:4: RuntimeWarning: divide by zero encountered in divide
  N = 1/P


To present the values nicely in this Jupyter notebook I'll use pandas to render the table with some appropriate string formatting on the numbers.

In [40]:
df = pd.DataFrame({'p':p, 'P': P, 'N': N, 'C': C, 'G1': G1, 'G': G, 'T': T, 'T_hours': T/60, 'T_days': T/60/24})

df['p'] = df['p'].map("{:.2f}".format)
df['P'] = df['P'].map("{:.3g}".format)
df['N'] = df['N'].map("{:.2f}".format)
df['C'] = df['C'].map("{:.2f}".format)
df['G1'] = df['G1'].map("{:.2f}".format)
df['G'] = df['G'].map("{:.2f}".format)
df['T'] = df['T'].map("{:.2f}".format)
df['T_hours'] = df['T_hours'].map("{:.2f}".format)
df['T_days'] = df['T_days'].map("{:.2f}".format)

df

,p,P,N,C,G1,G,T,T_hours,T_days
0,0.00,0,inf,inf,3.00,inf,inf,inf,inf
1,0.05,2.02e-14,49471586448457.00,494715864484570.00,3.16,156226062468810.34,1562260624688103.50,26037677078135.06,1084903211588.96
2,0.10,7.5e-11,13336889837.29,133368898372.90,3.33,44456299456.33,444562994563.32,7409383242.72,308724301.78
3,0.15,8.77e-09,114089950.84,1140899508.44,3.53,402670413.35,4026704133.54,67111735.56,2796322.31
4,0.20,2.48e-07,4034048.66,40340486.62,3.75,15127680.99,151276809.91,2521280.17,105053.34
5,0.25,3.21e-06,311410.04,3114100.42,4.00,1245638.56,12456385.61,207606.43,8650.27
6,0.30,2.53e-05,39514.41,395144.15,4.29,169345.75,1693457.51,28224.29,1176.01
7,0.35,0.000141,7087.20,70872.05,4.62,32708.28,327082.76,5451.38,227.14
8,0.40,0.000609,1642.91,16429.06,5.00,8212.44,82124.43,1368.74,57.03
9,0.45,0.00215,464.91,4649.06,5.45,2533.53,25335.30,422.25,17.59


## Conclusion
We've seen 4 approaches to solving this problem, namely:
1. Monte Carlo Simulation
2. Dynamic Programming
3. Linear Systems
4. Closed-Form Solutions / Analytics / Combinatorics / Stars-and-Bars

Along the way we've learned and used techniques from probability, sampling, dynamic programming, recursion, memoization, linear systems, matrices, combinatorics, and complexity theory.

This was an enjoyable.  
Oh, and, sounds like it would take a long time to win the prize in Brawliseum!

### Further Reading References and Links
1. Monte Carlo Simulation
   - https://en.wikipedia.org/wiki/Monte_Carlo_method
   - https://en.wikipedia.org/wiki/Sampling_(statistics)
2. Dynamic Programming
   - https://en.wikipedia.org/wiki/Dynamic_programming
   - https://en.wikipedia.org/wiki/Memoization
   - https://en.wikipedia.org/wiki/Recursion
   - https://en.wikipedia.org/wiki/Piecewise
3. Linear Systems
   - https://en.wikipedia.org/wiki/System_of_linear_equations
   - https://en.wikipedia.org/wiki/Sparse_matrix
   - https://en.wikipedia.org/wiki/Band_matrix
4. Combinatorics
   - https://en.wikipedia.org/wiki/Combinatorics
   - https://en.wikipedia.org/wiki/Stars_and_bars_(combinatorics)
   - https://en.wikipedia.org/wiki/Twelvefold_way
4. Probability
   - https://en.wikipedia.org/wiki/Probability
   - https://en.wikipedia.org/wiki/Expected_value

### Answers Summary
Using the backstory values
- $p = 50\%$ (since for each winner, there is also a loser)
- $n = 12$
- $k = 3$
- $c = \$ 10$
- $t = 10 \mathrm{mins}$

The answers are:
- The probability of winning the prize in a single admission $\mathrm{P} = 0.647%$.
- The expected number of admissions to win the prize $\mathrm{N} = 154.566$.
- The expected total cost to win the prize $\mathrm{C} = \$ 1545.66$.
- The expected number of games played in a single admission $\mathrm{G}_{1} = 5.983$.
- The expected number of games played to win the prize $\mathrm{G} = 924.792$.
- The expected total time it will take to win the prize $\mathrm{T} = 9247.925 \mathrm{mins} = 6 \mathrm{days} \, 10 \mathrm{hours} \, 8 \mathrm{mins}$.